In [7]:
import pickle
import re
import numpy as np

basis_file = '/home/anton/Documents/Tulane/Research/BSNIP/AngleBasis1.pkl'
demodir = '/home/anton/Documents/Tulane/Research/ImageNomer/data/anton/cohorts/BSNIP/'

basis = pickle.load(open(basis_file, 'rb'))

thetas = []
jitter = []
fc = []

demo = pickle.load(open(f'{demodir}/demographics.pkl', 'rb'))
subs = []

for subtask in basis:
    m = re.search('([^-]+)', subtask)
    sub = m.group(1)
    subs.append(sub)
    thetas.append(basis[subtask]['thetas'])
    jitter.append(basis[subtask]['jitter'])
    p = np.load(f'{demodir}/fc/{sub}_task-unk_fc.npy')
    fc.append(p)
    
thetas = np.stack(thetas)
jitter = np.stack(jitter)
fc = np.stack(fc)

print([a.shape for a in [thetas, jitter, fc]])

[(1244, 1, 264), (1244, 1, 264), (1244, 34716)]


In [8]:
def rmse(yhat, y):
    if isinstance(yhat, np.ndarray) or isinstance(yhat, int):
        f = np.mean
    else:
        f = torch.mean
    return f((y-yhat)**2)**0.5

def tops(thetas, jitter):
    t0 = np.expand_dims(thetas, 2)
    t1 = np.expand_dims(thetas, 3)
    j0 = np.expand_dims(jitter, 2)
    j1 = np.expand_dims(jitter, 3)
    ps = np.cos(t0-t1)*(j0*j1)
    a,b = np.triu_indices(264, 1)
    ps = ps[:,:,a,b]
    return ps
    
ps = tops(thetas, jitter)
aps = np.mean(ps, axis=1)
res = fc - aps

print(ps.shape)
print(aps.shape)
print(res.shape)

(1244, 1, 34716)
(1244, 34716)
(1244, 34716)


In [9]:
for sub,i in zip(subs,range(len(subs))):
    np.save(f'{demodir}/fc/{sub}_task-unkab1_fc.npy', aps[i])
    np.save(f'{demodir}/fc/{sub}_task-unkres1_fc.npy', res[i])
    
print('Complete')

Complete
